In [1]:
!pip install -Uq openai-agents 


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import os 
from dotenv import load_dotenv , find_dotenv # For Loading environment variables
from agents.run import RunConfig

# Load environment variables from .env file
load_dotenv(find_dotenv())

# set SECRET_KEY environment variable
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
model="gemini-2.0-flash"
base_url="https://generativelanguage.googleapis.com/v1beta/openai/"

In [3]:
from agents import AsyncOpenAI , OpenAIChatCompletionsModel , set_tracing_disabled

set_tracing_disabled(True)

provider = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url=base_url,
)

model = OpenAIChatCompletionsModel(
    openai_client=provider,
    model=model
)

config = RunConfig(
    model=model,
    tracing_disabled=True
)


# Local Context Example

In [28]:
from os import name
from agents import Agent , Runner , RunContextWrapper , function_tool
from dataclasses import dataclass
import asyncio

@dataclass
class UserInfo:
    name: str
    uuid: int
    hobies: list[str]


@function_tool
async def get_user_info(wrapper : RunContextWrapper[UserInfo] , greeting: str) -> str:
    """
    Greet the user with their name

    Args:
        greeting: special greeting for the user
    """
    name = wrapper.context.name
    return f"Name: {name} \nID: {wrapper.context.uuid} \nHobbies: {wrapper.context.hobies} \n{greeting}"

async def main():

    user_info = UserInfo(
        name="Abdullah" ,uuid=1 , hobies=["reading" , "Programming"]
    )

    agent = Agent[UserInfo](
        name='Assistant',
        instructions="Always greet the user using greet_user and welcome them to Panaversity",
        tools=[get_user_info]
    )

    result = await Runner.run(agent , "Hi?." , context=user_info , run_config=config)
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())


Welcome to Panaversity! Hello Abdullah, I am here to help you.

